# SQL

¡Bienvenido/a! En este Notebook trabajaremos con bases de datos relacionales y SQL. Así como el de Scraping, es principalmente, demostrativo.

Para eso usaremos la librería SQLite3, ya que es muy sencilla de configurar y no necesita instalar un servidor aparte. También tiene la ventaja de ser compatible con muchos lenguajes de programación. Esta librería suele venir instalada por defecto en Python, por lo que no es necesario instalarla.

### 1. Creando una Base de Datos

Importamos la librería y creamos un objeto **con**, con el cual nos conectaremos a la base de datos. Esto creará un archivo llamado "mi_primer_BD.db".

In [1]:
import pandas as pd
import numpy as np
import sqlite3

con = sqlite3.connect('mi_primer_BD.db')

Presta atención, te ha creado un nuevo archivo en la misma carpeta donde está este notebook. Esto lo hará únicamente si esa base de datos no existe previamente.

Si quieres trabajar con Python *puro* deberás crear el objeto **cursor**. En nuestro caso usaremos Pandas, que trae métodos para leer y modificar bases de datos SQL.

Igualmente te dejamos el código que deberías utilizar.

In [2]:
#CREAMOS LA BASE
#con = sqlite3.connect('mydatabase.db')
#CRREAMOS EL OBJETO CURSOR
#cursorObj = con.cursor()

Vamos a crear un dataframe que insertaremos en la base de datos como una tabla. Esta va a simular tener información sobre alumnos de Acamica.

In [3]:
# Creamos el diccionario
data_estudiantes = {'Nombre':['juan', 'esteban', 'nicolas', 'sofia', 'mariano', 'matias', 'paloma', 'valentina'], 'Edad':[30, 21, 19, 45, 27, 23, 29, 55], 'Carrera':['data science', 'data science', 'desarrollo web', 'data science', 'desarrollo web', 'UX', 'data science', 'desarrollo web']} 
  
# Creamos el DataFrame 
df_estudiantes = pd.DataFrame(data_estudiantes) 
df_estudiantes.head()

,Nombre,Edad,Carrera
0,juan,30,data science
1,esteban,21,data science
2,nicolas,19,desarrollo web
3,sofia,45,data science
4,mariano,27,desarrollo web


Ahora, cargamos el DataFrame `df_estudiantes` en la base de datos.

In [4]:
df_estudiantes.to_sql('estudiantes', con)

Para agregarle un poco más de complejidad a la base, vamos a crear una nueva tabla con algunos datos básicos de las carreras.

In [5]:
# Creamos el diccionario
data_carreras = {'Carrera':['data science', 'desarrollo web', 'UX', 'front end'],'Comisiones_cursando':[20, 15, 7, 9]} 
  
# Creamos el DataFrame 
df_carreras = pd.DataFrame(data_carreras) 

#cargamos el df en la BD
df_carreras.to_sql('carreras', con)

In [6]:
### Cerramos la conexión con la base de Datos

con.close()

¡Felicitaciones! Ya creaste tu primer base de datos de dos tablas.

### 2. Interactuando con la Base de Datos

Ahora vamos a interactuar con la base de datos y realizar algunas consultas básicas. Para hacerlo más realista, reinicia el kernel, así borramos cualquier información previa.

In [7]:
import pandas as pd
import numpy as np
import sqlite3

con = sqlite3.connect('mi_primer_BD.db')

Tal como viste en la bitácora, usaremos SELECT para buscar elementos. Mirá cada sentencia cómo es y qué hace. Verás que es fácilmente comprensible. Además, presta atención a que, una vez que hicimos el *query*, ya contamos con un DataFrame, ¡con el cual ya sabemos trabajar!

In [8]:
# Consultamos el nombre de las tablas que tiene nuestra BD
print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", con))

          name
0  estudiantes
1     carreras


In [9]:
# Consultamos la tabla estudiantes (el * es equivalente a TODO)
print(pd.read_sql("SELECT * FROM estudiantes;", con))

   index     Nombre  Edad         Carrera
0      0       juan    30    data science
1      1    esteban    21    data science
2      2    nicolas    19  desarrollo web
3      3      sofia    45    data science
4      4    mariano    27  desarrollo web
5      5     matias    23              UX
6      6     paloma    29    data science
7      7  valentina    55  desarrollo web


In [10]:
# Consultamos la tabla carreras
print(pd.read_sql("SELECT * FROM carreras;", con))

   index         Carrera  Comisiones_cursando
0      0    data science                   20
1      1  desarrollo web                   15
2      2              UX                    7
3      3       front end                    9


In [11]:
# Agregamos un condicional
print(pd.read_sql("SELECT Nombre FROM estudiantes WHERE Edad BETWEEN 20 and 30", con))

    Nombre
0     juan
1  esteban
2  mariano
3   matias
4   paloma


In [12]:
# Buscamos más información sobre estas personas
print(pd.read_sql("SELECT Nombre, Carrera FROM estudiantes WHERE Edad BETWEEN 20 and 30", con))

    Nombre         Carrera
0     juan    data science
1  esteban    data science
2  mariano  desarrollo web
3   matias              UX
4   paloma    data science


In [13]:
# Realizamos un pedido de la tabla Carreras
print(pd.read_sql("SELECT Carrera FROM carreras WHERE Comisiones_cursando > 10", con))

          Carrera
0    data science
1  desarrollo web


In [14]:
# Vamos a buscar el nombre de los estudiantes que cursan carreras con más de 15 comisiones abiertas (usaremos las dos tablas)
print(pd.read_sql("SELECT Nombre FROM estudiantes WHERE Carrera IN (SELECT Carrera FROM carreras WHERE Comisiones_cursando > 10)", con))

      Nombre
0       juan
1    esteban
2    nicolas
3      sofia
4    mariano
5     paloma
6  valentina


### 3. Haciendo cambios en la Base de Datos.

En esta sección haremos cambios en la base de datos. Para eso usaremos el método `to_sql` de Pandas: ya lo usaste para crear la base. [Aquí](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) te dejamos la documentación.

Por ejemplo, agregaremos filas a la tabla "estudiantes".

In [15]:
# Creamos el diccionario con los datos de los nuevos estudiantes
data_estudiantes2 = {'Nombre':['maria', 'celeste', 'martin', 'ana', 'ramiro', 'carlos'], 'Edad':[60, 17, 45, 29, 33, 23], 'Carrera':['desarrollo web', 'data science', 'UX', 'UX', 'data science', 'desarrollo web']} 

# Creamos el DataFrame 
df_estudiantes2 = pd.DataFrame(data_estudiantes2) 

# Cargamos el df en la BD
df_estudiantes2.to_sql('estudiantes', con, if_exists='append')

#imprimimos la base completa
print(pd.read_sql("SELECT * FROM estudiantes;", con))

    index     Nombre  Edad         Carrera
0       0       juan    30    data science
1       1    esteban    21    data science
2       2    nicolas    19  desarrollo web
3       3      sofia    45    data science
4       4    mariano    27  desarrollo web
5       5     matias    23              UX
6       6     paloma    29    data science
7       7  valentina    55  desarrollo web
8       0      maria    60  desarrollo web
9       1    celeste    17    data science
10      2     martin    45              UX
11      3        ana    29              UX
12      4     ramiro    33    data science
13      5     carlos    23  desarrollo web


**Para probar:**
1. Modifica la base de datos, agregando nuevas instancias o una tabla nueva.

In [16]:
data_carreras2 = {'Carrera':['back end', 'power bi'],'Comisiones_cursando':[12, 14]} 

In [17]:
df_carreras2 = pd.DataFrame(data_carreras2)

In [18]:
df_carreras2.to_sql('carreras', con, if_exists='append')

In [19]:
#imprimimos la base completa
print(pd.read_sql("SELECT * FROM carreras;", con))

   index         Carrera  Comisiones_cursando
0      0    data science                   20
1      1  desarrollo web                   15
2      2              UX                    7
3      3       front end                    9
4      0        back end                   12
5      1        power bi                   14


2. Intenta crear tu propia base de datos.

In [20]:
# Creamos el diccionario
data_alumnos = {'Nombre':['lucia', 'pablo', 'nicolas', 'abril', 'alma', 'lina'], 'Edad':[12, 11, 8, 11, 10, 9], 'Curso':['séptimo', 'sexto', 'tercero', 'sexto', 'quinto', 'cuarto']} 
  
# Creamos el DataFrame 
df_alumnos = pd.DataFrame(data_alumnos) 

#cargamos el df en la BD
df_alumnos.to_sql('alumnos', con)

In [21]:
# Creamos el diccionario
data_curso = {'Curso':['séptimo', 'sexto', 'tercero', 'sexto', 'quinto', 'cuarto'],'alumnos_cursando':[20, 15, 25, 28, 32, 18]} 
  
# Creamos el DataFrame 
df_curso = pd.DataFrame(data_curso) 

#cargamos el df en la BD
df_curso.to_sql('curso', con)

In [22]:
### Cerramos la conexión con la base de Datos

con.close()